#  Setup stuff:  Connectivity 

In [60]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [61]:

from katana import remote
from katana.remote import import_data

my_client=remote.Client()

print(my_client)


In [62]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [63]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, BwKBJYfN8y3jGkakJRQCpLWxkS729PeSRPMsQ5UVpMFF, 4>


#  Sample Cyphers: Vertices

In [ ]:

#  Cypher clauses
#
#     RETURN ..
#     ORDER BY ..
#     SKIP ..
#     LIMIT ..


In [64]:

#  KAT-11445   Mismatch columns and missing values

#  Return all nodes

#  l_query  = """
#     MATCH (n) 
#     RETURN n
#     """.format()
#  
#  l_return = my_graph.query(l_query)
#     #
#  print(l_return.head(10))

#          n.internal_id      n.labels  n.LABEL n.airport_code  n.airport_name n.type n.restaurant_code            n.restaurant_name
#     0                0     [Airport]  Airport            MKE       Milwaukee   node               NaN                          NaN
#     1                2  [Restaurant]      NaN            NaN             NaN   node               PAP  Pappadeauxs Seafood Kitchen
#     2                1     [Airport]  Airport            DEN          Denver   node               NaN                          NaN
#     3  562949953421313     [Airport]  Airport            SJC        San Jose   node               NaN                          NaN
#     4  562949953421312     [Airport]  Airport            ORD  Chicago O-Hare   node               NaN                          NaN


l_query  = """
   MATCH (n) 
   RETURN 
      n.internal_id AS my_id, 
      n.LABEL AS my_label, 
      n.airport_code AS airport_code, 
      n.airport_name AS airport_name, 
      n.restaurant_code AS restaurant_code, 
      n.restaurant_name AS restaurant_name
   """.format()

l_return = my_graph.query(l_query)
   #
print(l_return)



          0/? [?op/s]

  my_id my_label airport_code    airport_name restaurant_code              restaurant_name
0  None     None         None            None             PAP  Pappadeauxs Seafood Kitchen
1  None  Airport          ORD  Chicago O-Hare            None                         None
2  None  Airport          MKE       Milwaukee            None                         None
3  None  Airport          DEN          Denver            None                         None
4  None  Airport          SJC        San Jose            None                         None


In [65]:

#  Get a count of all nodes

display(my_graph.num_nodes())


l_query  = """
   MATCH (n) 
   RETURN COUNT(*) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]


print(l_count)
print(type(l_count))


          0/? [?op/s]

5

          0/? [?op/s]

5
<class 'numpy.int64'>


In [66]:

#  Get a count of all node types

l_query  = """
   MATCH (n) 
   RETURN COUNT(DISTINCT LABELS(n)) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]

print(l_count)


          0/? [?op/s]

2


In [67]:

#  List all node types

l_query  = """
   MATCH (n)
   RETURN DISTINCT LABELS(n) AS my_type
   """.format()

l_result = my_graph.query(l_query)


for l_each in l_result.itertuples():
   print(l_each.my_type)

print("--")


          0/? [?op/s]

['Restaurant']
['Airport']
--


In [68]:

#  List all node types with node counts

l_query  = """
   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))
   
    
print("")


l_query  = """
   MATCH (n) 
   WITH COUNT(n) AS tempCnt
   RETURN tempCnt
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))



          0/? [?op/s]

        label  cnt
0     Airport    4
1  Restaurant    1


None

          0/? [?op/s]

   tempCnt
0        5


None

#  Sample Cyphers: Edges

In [69]:

#  Count all edges

display(my_graph.num_edges())

l_query  = """
    MATCH (n) - [r] -> (m)
    RETURN COUNT(r) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]

print(l_count)


          0/? [?op/s]

8

          0/? [?op/s]

8


In [70]:

#  List all vertice types with counts

l_query  = """
   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))


l_query  = """
   MATCH (m)-[r]->(n) 
   WITH COUNT(r) AS tempCnt
   RETURN tempCnt
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))



          0/? [?op/s]

         temp  tempCnt
0  Attraction        1
1    FLIES_TO        7


None

          0/? [?op/s]

   tempCnt
0        8


None

#  Sample Cyphers: Properties

In [71]:

#  Return properties/keys from all vertices

my_graph.query("CALL graph.schema() RETURN *")


          0/? [?op/s]

,neighbor,nodeType,properties
0,,[Airport],"LABEL,airport_code,airport_name"
1,([Airport]::[FLIES_TO]),[Airport],"DISTANCE,NUM_HOPS,TYPE"
2,([Restaurant]::[Attraction]),[Airport],DRIVABLE
3,,[Restaurant],"restaurant_code,restaurant_name"


In [ ]:

#  Return properties/keys from all vertices

l_query  = """
   MATCH (n)
   RETURN DISTINCT labels(n) AS label, keys(n)
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))


          0/? [?op/s]

In [ ]:



l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT TYPE(r) AS my_type, keys(r)
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))








In [ ]:

#  Return properties/keys from all edges

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT (r.TYPE) AS my_type, keys(r) AS my_keys
   """.format()

l_result = my_graph.query(l_query)
   #
display(print(l_result))


#  Sample Cyphers: Diagnostics

In [ ]:

#  Also at,
#     http://localhost:8080/v2/widgets/katana-frontend

my_client.widgets().operations()

# my_client.widgets().databases()
# my_client.widgets().dashboard()                #  No-op currently


In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
# result.raw() went away
#
#  to pretty print a result as JSON, use ..

#  Just to get some data ..
#
l_result = my_graph.query("""

   MATCH (n) - [r] -> (m)
   RETURN DISTINCT type(r) AS type, keys(r)
   
   """)


import json

l_result_json = json.loads(l_result.to_json())
l_result_strn = json.dumps(l_result_json, indent=2)

print(l_result_strn)

In [ ]:
#  Produce a visual graph

result = my_graph.query("""

   MATCH (x) -[r]-> (a)
   RETURN x, r AS rel, a
   LIMIT 100 
   
   """,
   contextualize=True)

result.view()


#  More Cypher 

In [ ]:
#  Parameterized query, works with (Airports) ..

l_fname = "MKE"

l_query  = """

   MATCH (n: Airport) 
   WHERE n.id = '{0}'
   RETURN n.airport_name

   """.format(l_fname)

l_result1 = my_graph.query(l_query)

display(print(l_result1[0:5]))


In [ ]:

#  Regex query, works with (Airports) ..

#  0.4, returns zero data ..

l_fname = "/^M*/"

l_query  = """

   MATCH (n: Airport) 
   WHERE n.airport_name =~ '{0}'
   RETURN n.id, n.airport_name

   """.format(l_fname)

l_result1 = my_graph.query(l_query)

display(print(l_result1[0:5]))